In [3]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

import time
import numpy as np
import math


In [16]:
def CreateArray(TLeft,TRight,Ta,n):#This function creates the array according to a given equation For more info, go to page 787 in the class book
        array = np.zeros([n-2,n-2])
        b = np.zeros(len(array))
        deltaX = (10/(n-1))
        hPrime = 0.01
        
        for i in range(len(array)):
            if i == 0:
                array[i,i] = 2 + (hPrime * (deltaX**2))
                array[i,i+1] = -1
                b[i] = (hPrime*(deltaX**2) * Ta) + TLeft
                
            if i >0 and i < len(array) - 1:
                array[i,i-1] = -1
                array[i,i] = 2 + (hPrime * (deltaX**2))
                array[i,i+1] = -1    
                b[i] = hPrime * (deltaX**2) * Ta
                
            if i == len(array) - 1:
                array[i,i-1] = -1
                array[i,i] = 2 + (hPrime * (deltaX**2))                
                b[i] = (hPrime*(deltaX**2) * Ta) + TRight
        
        return array,b

In [17]:
def NaiveGauss(array,b):
    startTime = time.time()
    array = np.array(array , dtype = 'float')
    b = np.array(b , dtype = 'float')
    x = np.zeros(len(b))
    combined = np.zeros([len(b),len(b) + 1])
    
    for i in range(len(b)):
        if array[i,i] == 0:
            print('Naive Gauss only work sif the pivot does not equal 0!(reorganize if neccessary!)')
            return 'Error'
    
    for i in range(len(b)):
        for j in range(len(b)):
            combined[i,j] = array[i,j]
        combined[i,len(b)] = b[i]
    
    
    for i in range(len(b)):
        combined[i] = combined[i]/combined[i,i]
        for j in range(i + 1,len(b)):
            combined[j] -= (combined[i] * combined[j,i])
            
    array = np.delete(combined,len(b),1)
    b = np.delete(combined,np.s_[0:len(b)],1)

    for i in np.arange(len(b)-1,-1,-1):
        vals = array[i] * x
        x[i] = b[i] - sum(vals)
    
    totalTime = time.time() - startTime
    
    return x, totalTime

In [29]:
def DisplayTempArray(TLeft,TRight,Ta,n):
    TLeft,TRight,Ta,n = float(TLeft),float(TRight),float(Ta),int(n)
    
    array,b = CreateArray(TLeft,TRight,Ta,n)
    innerNodes, time1 = NaiveGauss(array,b)
    ApproximatedTemps = np.zeros(n)
    x = np.linspace(0,10,n)
    
    for i in range(n):
        if i == 0:
            ApproximatedTemps[i] = TLeft
        if i > 0 and i < n - 1:
            ApproximatedTemps[i] = innerNodes[i-1]
        if i == n - 1:
            ApproximatedTemps[i] = TRight
    
    plot_Temps = go.Scatter(name='Temperature Approximations', x=x, y=ApproximatedTemps)
        
    iplot([plot_Temps])
    
    return
        

In [30]:
w = interact(DisplayTempArray, TLeft = '40' , TRight = '200' , Ta = '20' , n = '6')


interactive(children=(Text(value='40', description='TLeft'), Text(value='200', description='TRight'), Text(val…